In [1]:
import pandas as pd
from datetime import date, datetime, timedelta
from functools import reduce
import requests
import io

In [2]:
## Jupyter options
pd.set_option('mode.chained_assignment', None)

# Configuration
today = date.today()
yesterday = date.today() - timedelta(days=1)

our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]
utlacodes_expressandstar = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
]
utlacodes_shropshirestar = [
    'E06000051', # Shropshire
    'E06000020', # Telford and Wrekin
]
utlacodes_blackcountry = [
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
]
utlacodes_blackcountryandbirmingham = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
]
utlacodes_sandwellandbirmingham = [
    'E08000025', # Birmingham
    'E08000028', # Sandwell
]
utlacodes_staffordshire = [
    'E10000028', # Staffordshire
]
utlacodes_wolverhampton = [
    'E08000031', # Wolverhampton
]
utlacodes_walsall = [
    'E08000030', # Walsall
]
utlacodes_dudley = [
    'E08000027', # Dudley
]
utlacodes_sandwell = [
    'E08000028', # Sandwell
]
utlacodes_birmingham = [
    'E08000025', # Birmingham
]
utlacodes_stoke = [
    'E06000021', # Stoke-on-Trent
]
utlacodes_staffordshireandstoke = [
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
]
utlacodes_worcestershire = [
    'E10000034', # Worcestershire
]

In [8]:
public_url = 'https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv'

datastr = requests.get(public_url, allow_redirects=True).text
data_file = io.StringIO(datastr)
df_csv = pd.read_csv(data_file)
df_filtered_columns = df_csv.copy().filter(
    regex='(^Area name$|^Area code$|^Specimen date$|^Cumulative lab-confirmed cases$)'
)
df_renamed_columns = df_filtered_columns.copy().rename(
    columns={
        'Area code': 'code',
        'Area name' : 'name',
        'Specimen date': 'date',
        'Cumulative lab-confirmed cases': 'cases',
    }
)
our_utlas = df_renamed_columns[df_renamed_columns.code.isin(our_utla_codes)]
our_utlas['date'] = pd.to_datetime(our_utlas['date']) # convert from strings
our_utlas = our_utlas.drop_duplicates(keep = 'last')
our_utlas['cases'] = our_utlas['cases'].astype(int)

In [9]:
our_utlas

,name,code,date,cases
10,Worcestershire,E10000034,2020-07-07,2347
15,Staffordshire,E10000028,2020-07-07,3666
113,Wolverhampton,E08000031,2020-07-07,1377
115,Walsall,E08000030,2020-07-07,1452
119,Sandwell,E08000028,2020-07-07,1628
...,...,...,...,...
47733,Staffordshire,E10000028,2020-03-03,2
47812,Staffordshire,E10000028,2020-03-02,1
47833,Birmingham,E08000025,2020-03-02,2
47885,Walsall,E08000030,2020-03-01,1


In [10]:
def get_latest_cases(code_group=False, df = our_utlas.copy()):
    # get the last and penultimate reporting dates
    last_day = df['date'].max()
    penultimate_day = last_day - timedelta(days=1)

    # remove all but the last two days of data
    df = df[(df['date'] == last_day) | (df['date'] == penultimate_day)]
    
    # group
    df = df.sort_values(by=['date'], ascending=False)
    df = df.groupby(['code','name','date'], as_index=False)['cases'].sum()
    df['diff'] = df.groupby('code')['cases'].diff().convert_dtypes(convert_integer=True)
    df = df[df['date'] == last_day]
    df = df.sort_values(by=['diff'], ascending=False)
    
    if code_group:
        df = df[df.code.isin(code_group)]
        df = df.groupby('date', as_index=False).sum()
        
    return df

In [11]:
get_latest_cases()

,code,name,date,cases,diff
1,E06000020,Telford and Wrekin,2020-07-07,595,0
3,E06000021,Stoke-on-Trent,2020-07-07,1443,0
5,E06000051,Shropshire,2020-07-07,1395,0
7,E08000025,Birmingham,2020-07-07,4805,0
13,E10000028,Staffordshire,2020-07-07,3666,0
8,E08000027,Dudley,2020-07-07,1301,<NA>
9,E08000028,Sandwell,2020-07-07,1628,<NA>
10,E08000030,Walsall,2020-07-07,1452,<NA>
11,E08000031,Wolverhampton,2020-07-07,1377,<NA>
14,E10000034,Worcestershire,2020-07-07,2347,<NA>


In [12]:
get_latest_cases(utlacodes_wolverhampton)

,date,cases,diff
0,2020-07-07,1377,0
